In [5]:
import polars as pl
import pandas as pd
from glob import glob
from tqdm import tqdm


In [2]:
parquets = glob("opm_parqs/*.parquet")


In [3]:
df_pl_res = [pl.read_parquet(fp) for fp in tqdm(parquets)]

100%|██████████| 16704/16704 [00:06<00:00, 2418.18it/s]


In [ ]:
df_merged = df_pl_res[0]
for df in tqdm(df_pl_res[1:]):
    df_merged = df_merged.join(df, on="date", how="outer")
# df_pl_joined = pl.concat(df_pl_res, how="horizontal")  # type: ignore


In [16]:
df_merged.write_parquet("./ch_data/merged.parquet")
df_merged.write_csv("./ch_data/merged.csv")
df_merged.shape

shape: (132, 16_705)
┌─────────────────────┬───────┬───────┬──────┬───┬──────┬──────┬──────┬──────┐
│ date                ┆ A.H.V ┆ A     ┆ AA   ┆ … ┆ SIGI ┆ SIGL ┆ SIGO ┆ SIGP │
│ ---                 ┆ ---   ┆ ---   ┆ ---  ┆   ┆ ---  ┆ ---  ┆ ---  ┆ ---  │
│ datetime[ns]        ┆ f64   ┆ f64   ┆ f64  ┆   ┆ f64  ┆ f64  ┆ f64  ┆ f64  │
╞═════════════════════╪═══════╪═══════╪══════╪═══╪══════╪══════╪══════╪══════╡
│ 2010-10-01 00:00:00 ┆ null  ┆ 10.4  ┆ null ┆ … ┆ null ┆ null ┆ null ┆ null │
│ 2011-01-01 00:00:00 ┆ null  ┆ 11.88 ┆ null ┆ … ┆ null ┆ null ┆ null ┆ null │
│ 2011-04-01 00:00:00 ┆ null  ┆ 12.91 ┆ null ┆ … ┆ null ┆ null ┆ null ┆ null │
│ 2011-07-01 00:00:00 ┆ null  ┆ 14.87 ┆ null ┆ … ┆ null ┆ null ┆ null ┆ null │
│ …                   ┆ …     ┆ …     ┆ …    ┆ … ┆ …    ┆ …    ┆ …    ┆ …    │
│ 1995-01-01 00:00:00 ┆ null  ┆ null  ┆ null ┆ … ┆ null ┆ null ┆ null ┆ null │
│ 1990-10-01 00:00:00 ┆ null  ┆ null  ┆ null ┆ … ┆ null ┆ null ┆ null ┆ null │
│ 1996-01-01 00:00:00 ┆ null  ┆

In [6]:
pddf = pd.read_csv("./ch_data/merged.csv")

In [7]:
pddf = pddf.dropna(axis=1,thresh=3).dropna(axis=0, thresh=1000).sort_values(by="date")

In [8]:
pddf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52 entries, 62 to 48
Columns: 9732 entries, date to SIGCY
dtypes: float64(9731), object(1)
memory usage: 3.9+ MB


In [19]:
df_merged[[s.name for s in df_merged if not (s.null_count() == df_merged.height)]]

date,A,AA,AAB.TO,AACAF,AACAY,AACS,AACTF,AADI,AAFRF,AAGEF,AAGRY,AAHIF,AAIRF,AAL,AALBF,AAMC,AAMMF,AAN,AANNF,AAOI,AAON,AAP,AAPL.NO,AAPL,AARTY,AASP,AAT,AAT.V,AATC,AATGF,AATP,AAV.TO,AAVVF,AAWH,AAWH.U.CX,AAYYY,…,SHTLF,SHUFF,SHUNF,SHVLF,SHVTF,SHW,SHWDF,SHWDY,SHWGF,SHWGY,SHWZ.NO,SHWZ,SHXWF,SHYF,SHZHY,SHZNF,SHZNY,SIA.TO,SIACF,SIARF,SIBN,SICRF,SICUF,SID,SIDU,SIEB,SIEGY,SIELF,SIELY,SIEN,SIETY,SIEVF,SIF,SIFY,SIG,SIGA,SIGCY
datetime[ns],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2010-10-01 00:00:00,10.4,null,83.6,31.66,31.66,-15.13,null,null,null,null,33.35,12.68,null,1.39,9.94,null,null,null,null,null,13.35,9.87,28.16,28.17,null,-21.01,36.12,-54.49,12.75,-55.21,null,67.63,67.86,null,null,11.26,…,null,null,15.6,30.07,6.86,9.56,4.87,4.87,null,null,null,null,null,2.41,24.63,46.42,46.42,null,null,null,null,19.19,null,32.8,null,-23.88,11.04,5.42,5.42,null,5.9,5.9,7.88,null,5.83,-66.21,null
2011-01-01 00:00:00,11.88,null,81.32,null,null,-10.19,null,null,null,null,34.5,12.82,null,1.65,null,null,null,null,null,null,11.98,9.82,28.98,29.02,null,-20.95,33.74,-46.22,7.33,-46.58,null,45.6,45.54,null,null,null,…,null,null,null,29.02,null,9.48,5.11,5.11,null,null,null,null,null,2.1,null,46.39,46.39,-5.7,null,null,null,19.84,null,32.89,null,-11.64,11.32,5.18,5.18,null,null,null,9.03,null,6.83,-101.9,null
2011-04-01 00:00:00,12.91,null,78.84,31.04,31.04,-9.39,null,null,null,null,35.22,14.29,null,0.43,10.36,null,null,null,null,null,10.66,10.01,30.37,30.43,null,-14.58,30.03,-48.96,5.29,-48.93,null,30.12,29.86,null,null,11.19,…,null,null,11.22,32.12,6.91,9.12,5.2,5.2,null,null,null,null,null,1.37,24.46,44.86,44.86,-5.79,null,null,null,23.75,null,31.06,null,-25.69,10.86,6.22,6.22,null,5.7,5.7,9.61,-3.79,8.48,-174.5,null
2011-07-01 00:00:00,14.87,null,78.04,null,null,-8.33,null,null,null,null,33.47,14.6,null,-0.86,null,null,null,null,null,null,10.86,10.41,31.19,31.22,null,-11.17,28.55,-63.15,-0.3,-62.23,null,32.02,31.32,null,null,null,…,null,null,null,30.46,null,8.87,5.88,5.88,null,null,null,null,null,1.35,null,52.4,52.4,-5.75,null,null,null,24.36,98.8,28.54,null,-19.44,10.52,6.76,6.76,null,null,null,10.19,-4.09,9.45,-257.3,null
2011-10-01 00:00:00,16.19,null,40.53,27.96,27.96,-5.07,null,null,null,null,29.55,14.67,null,-1.18,10.05,null,null,null,null,null,9.0,10.77,33.93,33.87,null,-5.9,27.79,-36.11,-5.55,-35.85,null,5.92,5.94,null,null,-32.48,…,null,null,7.79,31.13,6.37,8.84,5.55,5.55,null,null,null,null,null,0.56,23.56,53.42,53.42,-0.4,null,null,null,23.66,20.02,31.9,null,-21.71,9.29,7.54,7.54,null,5.89,5.89,9.37,null,9.93,-246.6,null
2012-01-01 00:00:00,16.8,null,null,null,null,-4.77,null,null,null,null,26.48,12.37,null,-0.52,null,null,null,null,null,null,9.37,11.29,35.75,35.69,null,-3.63,27.34,-36.46,-4.92,-36.25,null,3.74,3.78,null,null,null,…,null,2.96,null,31.32,null,9.08,5.64,5.64,null,null,null,null,null,0.46,null,54.47,54.47,-0.03,null,null,null,24.48,19.74,29.25,null,-12.69,8.97,8.04,8.04,null,null,null,8.51,null,10.16,-243.9,null
2012-04-01 00:00:00,17.17,null,null,28.26,28.26,-4.51,null,null,null,null,25.32,11.2,null,0.8,9.88,null,null,null,null,null,11.76,11.0,35.65,35.62,null,-5.04,28.14,-33.18,-7.28,-33.04,null,-2.51,-2.51,null,null,-33.83,…,null,null,5.03,29.78,4.86,9.6,5.23,5.23,null,null,null,null,null,1.86,22.83,56.02,56.02,1.75,null,null,null,21.81,17.06,10.85,null,-5.18,8.78,7.78,7.78,null,4.82,4.82,8.74,-2.87,10.41,-202.2,null
2012-07-01 00:00:00,16.92,null,null,null,null,-3.86,null,null,null,null,26.55,10.65,null,1.69,null,null,null,null,null,null,12.19,10.58,35.31,35.3,null,-7.01,28.95,-29.99,-3.36,-29.82,null,-5.42,-5.48,null,null,null,…,null,2.69,null,32.34,null,10.27,4.26,4.26,null,null,null,null,null,0.7,null,47.96,47.96,3.02,null,null,null,22.47,16.32,4.3,null,-3.49,8.56,7.84,7.84,nul